# Applied Data Science Capstone - Week 3

## Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

#### Author: Alejandro Del Campo

##### 
##### First of all, instal all the libraries that I will need in this step

In [1]:
import pandas as pd
#!pip install lxml
!pip install lxml bs4 html5lib
import lxml
#!conda install -c anaconda lxml

##### 
##### Next, copy the url and use the read_html command to read all de tables of the URL, then introduce the first table in the dataframe

In [2]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable=pd.read_html(page,flavor='html5lib')
toronto_df=wikitable[0]
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### The shape of the original dataframe

In [3]:
print('The  shape of dataframe is:',toronto_df.shape)

The  shape of dataframe is: (180, 3)


##### Let's drop the Postal Codes without any borough assigned

In [4]:
#Let's drop the row without Borough and Neighbourhood
df_to=toronto_df[toronto_df.Borough != 'Not assigned']
print('The new shape of dataframe is:',df_to.shape)
df_to=df_to.reset_index(drop=True)
df_to.head()

The new shape of dataframe is: (103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Step 2: Obtaining the coordinates of Toronto Neighbourhoods

In [5]:
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data
coord = pd.read_csv("Geospatial_data.csv", delimiter=",")

--2021-02-10 14:11:09--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2021-02-10 14:11:09--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-10 14:11:10--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-10 14:11:10--  https://ibm.box.com/public/static/9afzr83p

In [6]:
coord[0:5]

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
coord.shape

(103, 3)

#### I have in the dataframe "coord" the coordinates of all the Postal Codes, now i introduce them in the toronto neighbourhood dataframe

In [8]:
import numpy as np

df_to['Latitude'] = ""
df_to['Longitude'] = ""

for postcod in coord['Postal Code']:
    Lat = coord[coord['Postal Code']==postcod]['Latitude']
    Lat=float(Lat.values)
    Long = coord[coord['Postal Code']==postcod]['Longitude']
    Long=float(Long.values)
    
    reqd_Index = df_to[df_to['Postal Code']==postcod].index.tolist()
    index = reqd_Index[0]
    df_to.at[index,'Latitude']=Lat
    df_to.at[index,'Longitude']=Long

df_to.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6679,-79.5322
6,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
7,M3B,North York,Don Mills,43.7459,-79.3522
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789


In [9]:
df_to.shape

(103, 5)

### Step 3: Clustering Toronto Neighbourhoods

#### First: Import the libraries i will need

In [10]:
!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Find the coordinates of the center of Toronto

In [11]:
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


#### Plotting all the neighbourhoods of Toronto

In [12]:
# create map of Toronto using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_to['Latitude'], df_to['Longitude'], df_to['Borough'], df_to['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

In [41]:
# create map of Toronto using latitude and longitude values
map_to_bo = folium.Map(location=[latitude, longitude], zoom_start=10)

#Coloring the map by boroughs
no_borough=(int(len(df_to['Borough'].unique())))
x = np.arange(no_borough)
ys = [i + x + (i*x)**2 for i in range(no_borough)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
color=pd.DataFrame()
color['Borough']=df_to['Borough'].unique()
color['Color']=rainbow


# add markers to map
for lat, lng, borough, neighbourhood in zip(df_to['Latitude'], df_to['Longitude'], df_to['Borough'], df_to['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    colo =color[color['Borough']==borough]['Color']
    colo=colo.reset_index()
    c=colo['Color'][0]
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.7,
        parse_html=False).add_to(map_to_bo)
    
map_to_bo